# In-Distribution

In [ ]:
import os
from keras.models import load_model
from metrics import balanced_accuracy
import pandas as pd
import numpy as np
from keras import backend as K
from keras.utils import np_utils
from keras_numpy_backend import categorical_crossentropy, softmax
from image_iterator import ImageIterator
from lesion_classifier import LesionClassifier
from utils import preprocess_input as preprocess_input_trainset

temperature = 1000
noise_magnitude = 0.0014 # perturbation magnitude
num_classes = 8
image_data_format = K.image_data_format()

model_filepath = 'saved_models/DenseNet201_best_balanced_acc.hdf5'
model = load_model(filepath=model_filepath, custom_objects={'balanced_accuracy': balanced_accuracy(num_classes)})
model.summary()

In [ ]:
from image_iterator import ImageIterator
from lesion_classifier import LesionClassifier
from utils import preprocess_input as preprocess_input_trainset

# Load predicted results of validation set
df = pd.read_csv('predict_results/DenseNet201_best_balanced_acc.csv')
data_folder = 'C:/ISIC_2019'
derm_image_folder = os.path.join(data_folder, 'ISIC_2019_Training_Input')
df['path'] = df.apply(lambda row : os.path.join(derm_image_folder, row['image']+'.jpg'), axis=1)

In [ ]:
softmax_score_folder = 'softmax_scores'
if not os.path.exists(softmax_score_folder):
    os.makedirs(softmax_score_folder)
    
f1 = open(os.path.join(softmax_score_folder, 'confidence_Base_In.txt'), 'w')
f2 = open(os.path.join(softmax_score_folder, 'confidence_Base_Out.txt'), 'w')
g1 = open(os.path.join(softmax_score_folder, 'confidence_Our_In.txt'), 'w')
g2 = open(os.path.join(softmax_score_folder, 'confidence_Our_Out.txt'), 'w')

In [ ]:
# Calculating the confidence of the output, no perturbation added here, no temperature scaling used
for index, row in df.iterrows():
    softmax_probs = row[1:9]
    softmax_score = np.max(softmax_probs)
    f1.write("{}, {}, {}\n".format(temperature, noise_magnitude, softmax_score))
f1.close()

In [ ]:
# Reference https://stackoverflow.com/a/53650141/2437361

import keras
from keras import backend as K

# layer_dict = dict([(layer.name, layer) for layer in model.layers])

# an input layer to feed labels
y_true = keras.engine.input_layer.Input(shape=(num_classes,))

### Compute loss based on model's output, temperature Scaling, and true labels
# ce = K.mean(K.categorical_crossentropy(y_true, model.output))
scaled_dense_pred_output = model.get_layer('dense_pred').output / temperature
loss = K.categorical_crossentropy(y_true, K.softmax(scaled_dense_pred_output))

# Compute gradient of loss with respect to inputs
grad_loss = K.gradients(loss, model.inputs)

# Create a function to be able to run this computation graph
# The learning phase flag is a bool tensor (0 = test, 1 = train)
compute_perturbations = K.function(model.inputs + [y_true] + [K.learning_phase()],
                                   grad_loss)

# https://keras.io/getting-started/faq/#how-can-i-obtain-the-output-of-an-intermediate-layer
# get_dense_pred_layer_output = K.function([model.layers[0].input, K.learning_phase()],
#                                          [model.get_layer('dense_pred').output])
get_dense_pred_layer_output = K.function(model.inputs + [K.learning_phase()],
                                         [model.get_layer('dense_pred').output])

In [ ]:
# 0 = test, 1 = train
learning_phase = 0

generator = ImageIterator(
    image_paths=df['path'].tolist(),
    labels=np_utils.to_categorical(df['category'], num_classes=num_classes),
    augmentation_pipeline=LesionClassifier.create_aug_pipeline_val((224, 224)),
    batch_size=1,
    shuffle=False,
    rescale=None,
    preprocessing_function=preprocess_input_trainset,
    pregen_augmented_images=False,
    data_format=image_data_format
)

for i, (image, y_true_onehot) in enumerate(generator):
    print('y_true_onehot: ', y_true_onehot)
    
    perturbations = compute_perturbations([image, y_true_onehot, learning_phase])[0]
    print('perturbations: ', perturbations.shape)
    print(perturbations[0][0])
    
    # Get sign of perturbations
    perturbations = np.sign(perturbations)
    print(perturbations)
    
    # Normalize the perturbations to the same space of image
    # https://github.com/facebookresearch/odin/issues/5
    # TODO: I should use ISIC Training Set std = [0.2422, 0.2235, 0.2315]
    perturbations[0][0] = (perturbations[0][0])/(63.0/255.0)
    perturbations[0][1] = (perturbations[0][1])/(62.1/255.0)
    perturbations[0][2] = (perturbations[0][2])/(66.7/255.0)
    
    # Add perturbations to images
    perturbative_image = image - noise_magnitude * perturbations
    
    # Calculate the confidence after adding perturbations
    dense_pred_output = get_dense_pred_layer_output([perturbative_image, learning_phase])[0]
    dense_pred_output = dense_pred_output / temperature
    softmax_probs = softmax(dense_pred_output)
    softmax_score = np.max(softmax_probs)
    g1.write("{}, {}, {}\n".format(temperature, noise_magnitude, softmax_score))
    
    break
    
g1.close()